## 任务1：读取汽车问答数据

In [33]:
import json
import pdfplumber
import torch
import transformers
import jieba
import sklearn

questions = json.load(open("questions.json"))
print(questions[0])

pdf = pdfplumber.open("data.pdf")
pdf_content = []
for page_idx in range(len(pdf.pages)):
    pdf_content.append({
        'page': 'page_' + str(page_idx + 1),
        'content': pdf.pages[page_idx].extract_text()
    })
print(pdf_content[0])

{'question': '“前排座椅通风”的相关内容在第几页？', 'answer': '', 'reference': ''}
{'page': 'page_1', 'content': '欢迎\n感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。\n首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。\n本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际\n接收的车辆为准。\n如您有任何问题，或需要预约服务，请拨打电话4006-010101联系我们。您也可以开车前往Lynk&Co领克中心。\n在抵达之前，请您注意驾车安全。\n©领克汽车销售有限公司'}


## 任务2：文本嵌入与向量检索

### M3E

In [19]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('/Users/bytedance/huggingface.co/moka-ai/m3e-small')

question_sentences = [x['question'] for x in questions]
pdf_content_sentences = [x['content'] for x in pdf_content]

question_embeddings = model.encode(question_sentences, normalize_embeddings=True)
pdf_embeddings = model.encode(pdf_content_sentences, normalize_embeddings=True)

In [20]:
for query_idx, feat in enumerate(question_embeddings):
    score = feat @ pdf_embeddings.T
    max_score_page_idx = score.argsort()[-1] + 1
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx)

In [21]:
with open('submit.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

## 任务3：文本问答Promopt优化

### 对话 API

In [2]:
import time
import jwt
import requests

# 实际KEY，过期时间
def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split(".")
    except Exception as e:
        raise Exception("invalid apikey", e)

    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
        "timestamp": int(round(time.time() * 1000)),
    }
    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg": "HS256", "sign_type": "SIGN"},
    )

url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
headers = {
  'Content-Type': 'application/json',
  'Authorization': generate_token("{your key}", 3600)
}

data = {
    "model": "glm-3-turbo",
    "messages": [
        {"role": "user", "content": """我今天很不开心，给我讲一个笑话"""},
        {"role": "assistant", "content": """今天小明迟到了，但中了彩票"""},
        {"role": "user", "content": """这个笑话笑点在哪儿？"""},
    ]
}

response = requests.post(url, headers=headers, json=data)

print("Status Code", response.status_code)
print("JSON Response ", response.json())

Status Code 200
JSON Response  {'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': '这个笑话的笑点在于对比和意外转折。通常情况下，迟到是一个负面的事件，可能会导致一系列的不愉快后果，比如受到老师的批评或者错过重要的事情。然而，在这个笑话中，尽管小明迟到了，但他却中了彩票，这是一个完全出乎意料的正面结果，从而形成了一个意外的喜剧效果。这种转折让人感到意外和幽默，因为两种完全不同性质的事件被串联在一起，产生了一种反差效果。', 'role': 'assistant'}}], 'created': 1727338314, 'id': '20240926161151b827bc3bda634144', 'model': 'glm-3-turbo', 'request_id': '20240926161151b827bc3bda634144', 'usage': {'completion_tokens': 94, 'prompt_tokens': 32, 'total_tokens': 126}}


In [25]:
import time
import jwt
import requests
from tqdm import tqdm

import jieba, json, pdfplumber
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from rank_bm25 import BM25Okapi
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 实际KEY，过期时间
def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split(".")
    except Exception as e:
        raise Exception("invalid apikey", e)

    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
        "timestamp": int(round(time.time() * 1000)),
    }
    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg": "HS256", "sign_type": "SIGN"},
    )

def ask_glm(content):
    url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
    headers = {
      'Content-Type': 'application/json',
      'Authorization': generate_token("{your key}", 1000)
    }

    data = {
        "model": "glm-3-turbo",
        "messages": [{"role": "user", "content": content}]
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()

In [31]:
questions = json.load(open("questions.json"))
pdf = pdfplumber.open("data.pdf")
pdf_content = []
for page_idx in range(len(pdf.pages)):
    pdf_content.append({
        'page': 'page_' + str(page_idx + 1),
        'content': pdf.pages[page_idx].extract_text()
    })
pdf_content_words = [jieba.lcut(x['content']) for x in pdf_content]

model = SentenceTransformer('/Users/bytedance/huggingface.co/moka-ai/m3e-small')
question_sentences = [x['question'] for x in questions]
pdf_content_sentences = [x['content'] for x in pdf_content]

question_embeddings = model.encode(question_sentences, normalize_embeddings=True)
pdf_embeddings = model.encode(pdf_content_sentences, normalize_embeddings=True)

for query_idx in tqdm(range(len(question_embeddings))):
    feat = question_embeddings[query_idx]
    score = feat @ pdf_embeddings.T

    max_score_page_idx = score.argsort()[-1]
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx + 1)
    
    prompt = '''你是一个汽车专家，帮我结合给定的资料，回答下面的问题。如果问题无法从资料中获得，或无法从资料中进行回答，请无法回答问题。如果问题可以从资料中获得，则请逐步回答。
资料：{0}

问题：{1}
    '''.format(
        pdf_content[max_score_page_idx]['content'],
        questions[query_idx]["question"]
    )
    answer = ask_glm(prompt)['choices'][0]['message']['content']

    if '无法回答' in answer:
        answer = '结合给定的资料，无法回答问题。'
    
    questions[query_idx]['answer'] = answer

with open('submit.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

100%|█████████████████████████████████████████| 301/301 [20:37<00:00,  4.11s/it]


## 进阶：文本索引与答案检索

### TFIDF

In [6]:
import jieba
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

In [8]:
question_words = [' '.join(jieba.lcut(x['question'])) for x in questions]
pdf_content_words = [' '.join(jieba.lcut(x['content'])) for x in pdf_content]

'欢迎 \n 感谢您 选择 了 具有 优良 安全性 、 舒适性 、 动力性 和 经济性 的 Lynk & Co 领克 汽车 。 \n 首次 使用 前 请 仔细 、 完整 地 阅读 本手册 内容 ， 将 有助于 您 更好 地 了解 和 使用 车辆 。 \n 本手册 中 的 所有 资料 均 为 出版 时 的 最新 资料 ， 但本 公司 将 对 产品 进行 不断 的 改进 和 优化 ， 您 所购 的 车辆 可能 与 本手册 中 的 描述 有所不同 ， 请以 实际 \n 接收 的 车辆 为准 。 \n 如 您 有 任何 问题 ， 或 需要 预约 服务 ， 请拨 打电话 4006 - 010101 联系 我们 。 您 也 可以 开车 前往 Lynk & Co 领克 中心 。 \n 在 抵达 之前 ， 请 您 注意 驾车 安全 。 \n © 领克 汽车 销售 有限公司'

In [9]:
tfidf = TfidfVectorizer()
tfidf.fit(question_words + pdf_content_words)

question_feat = tfidf.transform(question_words)
pdf_content_feat = tfidf.transform(pdf_content_words)

question_feat = normalize(question_feat)
pdf_content_feat = normalize(pdf_content_feat)

In [12]:
for query_idx, feat in enumerate(question_feat):
    score = feat @ pdf_content_feat.T
    score = score.toarray()[0]
    max_score_page_idx = score.argsort()[-1] + 1

    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx)

0
  (0, 4181)	0.44486360335226943
  (0, 3484)	0.5425856162365112
  (0, 3347)	0.32929184994563093
  (0, 2103)	0.29322556256895066
  (0, 1273)	0.32929184994563093
  (0, 1136)	0.4526033009607539
1
  (0, 4009)	0.12234907061949699
  (0, 3347)	0.25554365791487615
  (0, 2323)	0.3650903935752825
  (0, 2103)	0.2275547750889947
  (0, 1884)	0.17370114868265363
  (0, 1685)	0.39307927640116397
  (0, 1508)	0.1622433475779391
  (0, 1136)	0.35123828036119065
  (0, 1107)	0.3650903935752825
  (0, 1050)	0.26939577112896806
  (0, 596)	0.4409265876243212
2
  (0, 3699)	0.4138222180192917
  (0, 3484)	0.5683173764225756
  (0, 3347)	0.34490829583079613
  (0, 2305)	0.2751798300156953
  (0, 1276)	0.45850347751935355
  (0, 1002)	0.3175774256020776
3
  (0, 4118)	0.4998631240187915
  (0, 3699)	0.3777975018827763
  (0, 2525)	0.271820416362542
  (0, 2305)	0.25122443363736974
  (0, 1276)	0.4185890966427089
  (0, 596)	0.5433130020358671
4
  (0, 3838)	0.3492379996016712
  (0, 3793)	0.37348111548006924
  (0, 3744)	0.3074

In [11]:
with open('submit.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

### BM25

In [52]:
# !pip install rank_bm25
from rank_bm25 import BM25Okapi

pdf_content_words = [jieba.lcut(x['content']) for x in pdf_content]
bm25 = BM25Okapi(pdf_content_words)

In [64]:
for query_idx in range(len(questions)):
    doc_scores = bm25.get_scores(jieba.lcut(questions[query_idx]["question"]))
    max_score_page_idx = doc_scores.argsort()[-1] + 1
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx)

In [65]:
with open('submit.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

### LlamaIndex

In [3]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
#5步搭建简易RAG pipeline
# LlamaIndex
#1.加载并解析文档
documents = SimpleDirectoryReader("./data.pdf").Load_data()
#2.Embed并创建索引
index = VectorStoreindex.from_documents(documents,"text","title")
#3.基于1ndex生成query引擎
query_engine = index.as_query_engine()
#4.查询
response = query_engine.query("问题：事件数据记录系统（EDR）中的数据是否可以被黑客利用进行恶意攻击？")
#5,输出结果
print(response)

ValueError: Directory ./data.pdf does not exist.